In [1]:
## DESCRIPTION
# meteo_precipitation_passirio_cells_rea11_kr11
# meteo_precipitation_passirio_monthly_cellsBoxplot_kr11-rea11

In [2]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [3]:
from lib import *

In [4]:
wdir = "/home/daniele/documents/github/ftt01/phd/projects/era5_evaluation/"
current = DataCollector(configPath=wdir + "etc/conf/")

In [5]:
## SETUP
basin = 'passirio'
# for output name
basin_str = 'passirio'

start_date_str = "2010-01-01T00:00:00"
end_date_str = "2019-12-31T23:00:00"
start_date = dt.datetime.strptime( start_date_str, '%Y-%m-%dT%H:%M:%S' )
end_date = dt.datetime.strptime( end_date_str, '%Y-%m-%dT%H:%M:%S' )

In [6]:
## REANALYSIS DATASET ##
for el in current.sim_config:
    if el['basin'] == basin:
        meteo_stations_rea_path =  el["sim_path"] + el["meteo_path"] + "reanalysis/observations/precipitation/"
        # print(meteo_stations_rea_path)
        precipitation_df_rea = pd.DataFrame()

        p_file_tot = glob.glob( meteo_stations_rea_path + '*.csv')
        
        for p_file in p_file_tot:
            data_tot = pd.read_csv(p_file, header=0, index_col=0, \
                skiprows=4, parse_dates=True, names=[ os.path.basename(p_file)[:-4] ])
            data_tot = data_tot[start_date:end_date]
            data_tot[data_tot == -999] = None
            precipitation_df_rea = pd.concat([precipitation_df_rea, data_tot],axis=1)

# precipitation_rea_sum = precipitation_df_rea.resample('MS').sum()
precipitation_rea_spatial_mean = precipitation_df_rea.mean(axis=1)

In [7]:
## KRIGING 11X8 DATASET ##
for el in current.sim_config:
    if el['basin'] == basin:
        meteo_stations_kr11_path =  el["sim_path"] + el["meteo_path"] + "kriging/observations/11x8/precipitation/"
           
        precipitation_df_kr11 = pd.DataFrame()

        p_file_tot = glob.glob( meteo_stations_kr11_path + '*.csv')
        
        for p_file in p_file_tot:
            data_tot = pd.read_csv(p_file, header=0, index_col=0, \
                skiprows=4, parse_dates=True, names=[ os.path.basename(p_file)[:-4] ])
            data_tot = data_tot[start_date:end_date]
            data_tot[data_tot == -999] = None
            precipitation_df_kr11 = pd.concat([precipitation_df_kr11, data_tot],axis=1)

precipitation_kr11_spatial_mean = precipitation_df_kr11.mean(axis=1)

In [8]:
# boxplot monthly bias
precipitation_bias = precipitation_rea_spatial_mean - \
    precipitation_kr11_spatial_mean
precipitation_bias_monthly_sum = precipitation_bias.resample('MS').sum()

output_path = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/monthly/" \
    + "meteo_precipitation_" + basin_str + \
    "_monthly_spatial_mean_cells_boxplot_rea-kr11." + output_format
createBoxPlot(precipitation_bias_monthly_sum,  "Time $[month]$", 'Prec. bias $[mm/month]$',
              output_path, label="(a)", period='MS', scale_factor=0.5, output_format=output_format, my_dpi=50)

output_path_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/monthly/" \
    + "meteo_precipitation_" + basin_str + \
    "_monthly_spatial_mean_cells_boxplot_rea-kr11_HD." + output_format
createBoxPlot(precipitation_bias_monthly_sum,  "Time $[month]$", 'Prec. bias $[mm/month]$', output_path_hd,
              label="(a)", period='MS', scale_factor=0.5, output_format=output_format, my_dpi=600)

In [9]:
# spatial ECDF - hourly
precipitation_rea_spatial_ecdf = evaluateECDF( precipitation_rea_spatial_mean )
precipitation_kr11_spatial_ecdf = evaluateECDF( precipitation_kr11_spatial_mean )

plots = []

plt_conf = {}
plt_conf["label"] = "REA11x8"
plt_conf["color"] = '#e66101'
plots.append( (precipitation_rea_spatial_ecdf, plt_conf) )

plt_conf = {}
plt_conf["label"] = "KR11x8"
plt_conf["color"] = '#5e3c99'
plots.append( (precipitation_kr11_spatial_ecdf, plt_conf) )

output_file = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/hourly/meteo_" \
    + 'precipitation_passirio_hourly_spatialECDF.' + output_format
createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file, \
    scale_factor=0.5, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/hourly/meteo_" \
    + 'precipitation_passirio_hourly_spatialECDF_HD.' + output_format
createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file_hd, \
    scale_factor=0.5, my_dpi=600)

output_file = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/hourly/meteo_" \
    + 'precipitation_passirio_hourly_spatialECDF_log.' + output_format
createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file, \
    xscale='log', scale_factor=0.5, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/hourly/meteo_" \
    + 'precipitation_passirio_hourly_spatialECDF_log_HD.' + output_format
createPlot( plots, "Precipitation $[mm/hour]$", "ECDF", output_file_hd, \
    xscale='log', scale_factor=0.5, my_dpi=600)

In [10]:
# spatial ECDF - monthly
precipitation_rea_spatial_ecdf = evaluateECDF( precipitation_rea_spatial_mean.resample('MS').sum() )
precipitation_kr11_spatial_ecdf = evaluateECDF( precipitation_kr11_spatial_mean.resample('MS').sum() )

plots = []

plt_conf = {}
plt_conf["label"] = "REA11x8"
plt_conf["color"] = '#e66101'
plots.append( (precipitation_rea_spatial_ecdf, plt_conf) )

plt_conf = {}
plt_conf["label"] = "KR11x8"
plt_conf["color"] = '#5e3c99'
plots.append( (precipitation_kr11_spatial_ecdf, plt_conf) )

output_file = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/monthly/meteo_" \
    + 'precipitation_passirio_monthly_spatialECDF.' + output_format
createPlot( plots, "Precipitation $[mm/month]$", "ECDF", output_file, \
    scale_factor=0.5, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/monthly/meteo_" \
    + 'precipitation_passirio_monthly_spatialECDF_HD.' + output_format
createPlot( plots, "Precipitation $[mm/month]$", "ECDF", output_file_hd, \
    scale_factor=0.5, my_dpi=600)

output_file = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/monthly/meteo_" \
    + 'precipitation_passirio_monthly_spatialECDF_log.' + output_format
createPlot( plots, "Precipitation $[mm/month]$", "ECDF", output_file, \
    xscale='log', scale_factor=0.5, my_dpi=50)

output_file_hd = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/monthly/meteo_" \
    + 'precipitation_passirio_monthly_spatialECDF_log_HD.' + output_format
createPlot( plots, "Precipitation $[mm/month]$", "ECDF", output_file_hd, \
    xscale='log', scale_factor=0.5, my_dpi=600)

In [11]:
rea = precipitation_rea_spatial_ecdf
kr = precipitation_kr11_spatial_ecdf
bias_min = np.array(rea.loc[(rea['cdf'] <= 5)].index).mean() - np.array(kr.loc[(kr['cdf'] <= 5)].index).mean()
print(bias_min)

14.257851851851854


In [12]:
rea = precipitation_rea_spatial_ecdf
kr = precipitation_kr11_spatial_ecdf
bias_median = np.array(rea.loc[(rea['cdf'] >= 49.5) & (rea['cdf'] <= 50.5)].index).mean() - np.array(kr.loc[(kr['cdf'] >= 49.5) & (kr['cdf'] <= 50.5)].index).mean()
print(bias_median)

36.55283333333334


In [13]:
rea = precipitation_rea_spatial_ecdf
kr = precipitation_kr11_spatial_ecdf
bias_max = np.array(rea.loc[(rea['cdf'] >= 95)].index).mean() - np.array(kr.loc[(kr['cdf'] >= 95)].index).mean()
print(bias_max)

33.19244444444442


In [14]:
precipitation_bias = precipitation_rea_spatial_mean - precipitation_kr11_spatial_mean

#-------------------boxplot seasonal--------------------     
precipitation_bias_jan=precipitation_bias.loc[(precipitation_bias.index.month==1)]
precipitation_bias_feb=precipitation_bias.loc[(precipitation_bias.index.month==2)]
precipitation_bias_mar=precipitation_bias.loc[(precipitation_bias.index.month==3)]
precipitation_bias_apr=precipitation_bias.loc[(precipitation_bias.index.month==4)]
precipitation_bias_may=precipitation_bias.loc[(precipitation_bias.index.month==5)]
precipitation_bias_jun=precipitation_bias.loc[(precipitation_bias.index.month==6)]
precipitation_bias_jul=precipitation_bias.loc[(precipitation_bias.index.month==7)]
precipitation_bias_aug=precipitation_bias.loc[(precipitation_bias.index.month==8)]
precipitation_bias_sep=precipitation_bias.loc[(precipitation_bias.index.month==9)]
precipitation_bias_oct=precipitation_bias.loc[(precipitation_bias.index.month==10)]
precipitation_bias_nov=precipitation_bias.loc[(precipitation_bias.index.month==11)]
precipitation_bias_dec=precipitation_bias.loc[(precipitation_bias.index.month==12)]

precipitation_bias_w=pd.concat([precipitation_bias_jan,precipitation_bias_feb,precipitation_bias_mar])
precipitation_bias_sp=pd.concat([precipitation_bias_apr,precipitation_bias_may,precipitation_bias_jun])
precipitation_bias_su=pd.concat([precipitation_bias_jul,precipitation_bias_aug,precipitation_bias_sep])
precipitation_bias_a=pd.concat([precipitation_bias_oct,precipitation_bias_nov,precipitation_bias_dec])

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_winter.' + output_format
createBoxPlot( precipitation_bias_w, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_winter_HD.' + output_format
createBoxPlot( precipitation_bias_w, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_spring.' + output_format
createBoxPlot( precipitation_bias_sp, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_spring_HD.' + output_format
createBoxPlot( precipitation_bias_sp, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_summer.' + output_format
createBoxPlot( precipitation_bias_su, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_summer_HD.' + output_format
createBoxPlot( precipitation_bias_su, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

output_file = current.config["output_path"] + "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_autumn.' + output_format
createBoxPlot( precipitation_bias_a, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file, output_format=output_format, period="H", scale_factor=0.5, my_dpi=50 )

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/seasonal/" \
    + 'meteo_precipitation_' + basin_str + '_hourly_boxplot_spatial_mean_cells_rea-kr11_autumn_HD.' + output_format
createBoxPlot( precipitation_bias_a, "Time $[hour]$", "Prec. $[mm/hour]$", \
    output_file_hd, output_format=output_format, period="H", scale_factor=0.5, my_dpi=600 )

In [15]:
# exceedance probability plot - hourly
precipitation_rea_fd_mod = precipitation_rea_spatial_mean
precipitation_rea_fd_mod[precipitation_rea_fd_mod<=0.01] = 0
precipitation_rea_fd_mod.dropna(inplace=True)

precipitation_kr11_fd_mod = precipitation_kr11_spatial_mean
precipitation_kr11_fd_mod[precipitation_kr11_fd_mod<=0.01] = 0
precipitation_kr11_fd_mod.dropna(inplace=True)

precipitation_rea_spatial_fd = flowDuration( precipitation_rea_fd_mod )
precipitation_kr11_spatial_fd = flowDuration( precipitation_kr11_fd_mod )

plots = []

### kriging 11x8 plot ###
plt_conf = {}
plt_conf["label"] = 'KR11x8'
plt_conf["color"] = '#5e3c99'
# plt_conf["marker"] = 'o'
plots.append( (precipitation_kr11_spatial_fd, plt_conf) )

### rea 11x8 plot ###
plt_conf = {}
plt_conf["label"] = 'REA11x8'
plt_conf["color"] = '#e66101'
# plt_conf["marker"] = 'o'
plots.append( (precipitation_rea_spatial_fd, plt_conf) )

from matplotlib.ticker import FormatStrFormatter
# y_major_locator=FormatStrFormatter('%.2f')
y_major_locator=None

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/meteo_" + 'precipitation_AA_hourly_spatialFD_log.' + output_format
createPlot( plots, "Exceedance Probability", "Prec. $[mm/hour]$", output_file, xticks=[0,10,20,30,40,50,60,70,80,90,100], x_rot=90, \
label="(a)", y_major_formatter=y_major_locator, yscale='log', y_lim_min=0.01, y_lim_max=15, \
    height=160, scale_factor=0.35, bbox_to_anchor=None, my_dpi=50)

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/meteo_" + 'precipitation_AA_hourly_spatialFD_log_HD.' + output_format
createPlot( plots, "Exceedance Probability", "Prec. $[mm/hour]$", output_file_hd, xticks=[0,10,20,30,40,50,60,70,80,90,100], x_rot=90, \
label="(a)", y_major_formatter=y_major_locator, yscale='log', y_lim_min=0.01, y_lim_max=15, \
    height=160, scale_factor=0.35, bbox_to_anchor=None, my_dpi=600)

#############

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/meteo_" + 'precipitation_AA_hourly_spatialFD_symlog.' + output_format
createPlot( plots, "Exceedance Probability", "Prec. $[mm/hour]$", output_file, label="(a)",\
    y_major_formatter=y_major_locator, yscale='symlog', height=160, scale_factor=0.35, bbox_to_anchor=None, my_dpi=50)

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/meteo_" + 'precipitation_AA_hourly_spatialFD_symlog_HD.' + output_format
createPlot( plots, "Exceedance Probability", "Prec. $[mm/hour]$", output_file_hd, label="(a)",\
    y_major_formatter=y_major_locator, yscale='symlog', height=160, scale_factor=0.35, bbox_to_anchor=None, my_dpi=600)

############

output_file = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/meteo_" + 'precipitation_AA_hourly_spatialFD.' + output_format
createPlot( plots, "Exceedance Probability", "Prec. $[mm/hour]$", output_file, label="(a)",\
    y_major_formatter=y_major_locator, height=160, scale_factor=0.35, bbox_to_anchor=None, my_dpi=50)

output_file_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/hourly/meteo_" + 'precipitation_AA_hourly_spatialFD_HD.' + output_format
createPlot( plots, "Exceedance Probability", "Prec. $[mm/hour]$", output_file_hd, label="(a)",\
    y_major_formatter=y_major_locator, height=160, scale_factor=0.35, bbox_to_anchor=None, my_dpi=600)

In [16]:
# plots = []

# plt_conf = {}
# plt_conf["label"] = 'KR11x8'
# plt_conf["color"] = 'orange'
# plots.append( (precipitation_kr11_mean, plt_conf) )

# plt_conf = {}
# plt_conf["label"] = 'REA11x8'
# plt_conf["color"] = '#e66101'
# plots.append( (precipitation_rea_mean, plt_conf) )

# outfile = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/monthly/" \
#     + "meteo_precipitation_" + basin_str + "_cells_rea11_kr11." + output_format
# createPlot( plots, "Time $[month]$", 'Precipitation $[mm/month]$', outfile, my_dpi=50 )

# outfile_hd = current.config["output_path"] +  "meteo/" + basin_str + "/precipitation/" + "/monthly/" \
#     + "meteo_precipitation_" + basin_str + "_cells_rea11_kr11_hd." + output_format
# createPlot( plots, "Time $[month]$", 'Precipitation $[mm/month]$', outfile_hd, my_dpi=600 )

# #######################################################################################################

In [17]:
precipitation_bias_monthly = precipitation_bias.resample("MS").sum()

print(f"Min: {precipitation_bias_monthly.min()}")
print(f"Max: {precipitation_bias_monthly.max()}")
print(f"Mean: {precipitation_bias_monthly.mean()}")

Min: -65.11
Max: 111.66655555555556
Mean: 33.433026851851864


In [29]:
result_string = ""
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for i, month in enumerate(months, start=1):
    query_result = precipitation_bias.loc[precipitation_bias.index.month == i]
    monthly_sum = query_result.sum()  # Calculate the sum for the month
    result_string += f"Total precipitation bias for {month}: {round(monthly_sum/10,2)}\n"

print(result_string)

Total precipitation bias for Jan: 31.13
Total precipitation bias for Feb: 28.3
Total precipitation bias for Mar: 24.59
Total precipitation bias for Apr: 28.51
Total precipitation bias for May: 43.54
Total precipitation bias for Jun: 42.15
Total precipitation bias for Jul: 67.91
Total precipitation bias for Aug: 51.85
Total precipitation bias for Sep: 23.53
Total precipitation bias for Oct: 25.68
Total precipitation bias for Nov: 7.58
Total precipitation bias for Dec: 26.43

